# INFO 4271 - Group Project

Issued: June 17, 2025

Due: July 21, 2025

Please submit a link to your code repository (with a branch that does not change anymore after the submission deadline) and your 4-page report via email to carsten.eickhoff@uni-tuebingen.de by the due date. One submission per team.

---

# 1. Web Crawling & Indexing
Crawl the web to discover **English content related to Tübingen**. The crawled content should be stored locally. If interrupted, your crawler should be able to re-start and pick up the crawling process at any time.

In [13]:
%load_ext autoreload
%autoreload 2

import urllib.robotparser as urobot
import urllib.request
from bs4 import BeautifulSoup

url = "http://google.com/"
rp = urobot.RobotFileParser()
rp.set_url(url + "robots.txt")
rp.read()
if rp.can_fetch("*", url + "catalogs/about"):
    site = urllib.request.urlopen(url)
    sauce = site.read()
    soup = BeautifulSoup(sauce, "html.parser")
    actual_url = site.geturl()[:site.geturl().rfind('/')]
    
    my_list = soup.find_all("a", href=True)
    for i in my_list:
        # rather than != "#" you can control your list before loop over it
        if i != "#":
            newurl = str(actual_url)+"/"+str(i)
            try:
                if rp.can_fetch("*", newurl):
                    site = urllib.request.urlopen(newurl)
                    # do what you want on each authorized webpage
            except:
                pass
else:
    print("cannot scrape")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cannot scrape


In [ ]:
%load_ext autoreload
%autoreload 2

import project

start_urls = ["https://duckduckgo.com/?q=tübingen",
              "https://en.wikipedia.org/wiki/T%C3%BCbingen",
              'https://www.mygermanyvacation.com/best-things-to-do-and-see-in-tubingen-germany/',
              'https://www.britannica.com/place/Tubingen-Germany',
              'https://www.germany.travel/en/cities-culture/tuebingen.html',]
crawler = project.Crawler(urls=start_urls)
crawler.run()

2025-06-28 19:01:52,136 INFO:Cleaning up old proxies...
2025-06-28 19:01:52,136 INFO:Fetching proxies list...
2025-06-28 19:01:52,349 INFO:Found 300 candidate proxies. Testing...
2025-06-28 19:02:17,047 INFO:27 proxies valid after testing.
2025-06-28 19:02:18,263 INFO:Visited 1 pages (Visited https://www.britannica.com/place/Tubingen-Germany, english=True, keywords_found=True)
2025-06-28 19:02:19,130 INFO:Visited 2 pages (Visited https://en.wikipedia.org/wiki/T%C3%BCbingen, english=True, keywords_found=True)
2025-06-28 19:02:19,825 INFO:Visited 3 pages (Visited https://www.mygermanyvacation.com/best-things-to-do-and-see-in-tubingen-germany/, english=True, keywords_found=True)


In [7]:
{p for p, d in crawler.visited_pages.items() if d["keywords_found"]}

{'https://af.wikipedia.org/wiki/T%C3%BCbingen',
 'https://als.wikipedia.org/wiki/T%C3%BCbingen',
 'https://an.wikipedia.org/wiki/Tubinga',
 'https://ang.wikipedia.org/wiki/Tybingen',
 'https://ar.wikipedia.org/wiki/%D8%AA%D9%88%D8%A8%D9%8A%D9%86%D8%BA%D9%86',
 'https://arz.wikipedia.org/wiki/%D8%AA%D9%88%D8%A8%D9%8A%D9%86%D8%AC%D9%8A%D9%86',
 'https://azb.wikipedia.org/wiki/%D8%AA%D9%88%D8%A8%DB%8C%D9%86%D9%82%D9%86',
 'https://ban.wikipedia.org/wiki/T%C3%BCbingen',
 'https://be-tarask.wikipedia.org/wiki/%D0%A2%D1%83%D0%B1%D1%96%D0%BD%D0%B3%D0%B5%D0%BD',
 'https://be.wikipedia.org/wiki/%D0%A6%D1%8E%D0%B1%D1%96%D0%BD%D0%B3%D0%B5%D0%BD',
 'https://bg.wikipedia.org/wiki/%D0%A2%D1%8E%D0%B1%D0%B8%D0%BD%D0%B3%D0%B5%D0%BD',
 'https://br.wikipedia.org/wiki/T%C3%BCbingen',
 'https://ca.wikipedia.org/wiki/T%C3%BCbingen',
 'https://ce.wikipedia.org/wiki/%D0%A2%D1%83%D1%8C%D0%B1%D0%B8%D0%BD%D0%B3%D0%B5%D0%BD',
 'https://ceb.wikipedia.org/wiki/T%C3%BCbingen_(munisipyo)',
 'https://commons.wikimedia

# 2. Query Processing 
Process a textual query and return the 100 most relevant documents from your index. Please incorporate **at least one retrieval model innovation** that goes beyond BM25 or TF-IDF. Please allow for queries to be entered either individually in an interactive user interface (see also #3 below), or via a batch file containing multiple queries at once. The batch file (see `queries.txt` for an example) will be formatted to have one query per line, listing the query number, and query text as tab-separated entries. An example of the batch file for the first two queries looks like this:

```
1   tübingen attractions
2   food and drinks
```

In [ ]:
#Retrieve documents relevnt to a query. You need (at least) two parameters:
	#query: The user's search query
	#index: The location of the local index storing the discovered documents.
def retrieve(query, index):
    #TODO: Implement me
	pass

# 3. Search Result Presentation
Once you have a result set, we want to return it to the searcher in two ways: a) in an interactive user interface. For this user interface, please think of **at least one innovation** that goes beyond the traditional 10-blue-links interface that most commercial search engines employ. b) as a text file used for batch performance evaluation. The text file should be formatted to produce one ranked result per line, listing the query number, rank position, document URL and relevance score as tab-separated entries. An example of the first three lines of such a text file looks like this:

```
1   1   https://www.tuebingen.de/en/3521.html   0.725
1   2   https://www.komoot.com/guide/355570/castles-in-tuebingen-district   0.671
1   3   https://www.unimuseum.uni-tuebingen.de/en/museum-at-hohentuebingen-castle   0.529
...
1   100 https://www.tuebingen.de/en/3536.html   0.178
2   1   https://www.tuebingen.de/en/3773.html   0.956
2   2   https://www.tuebingen.de/en/4456.html   0.797
...
```

In [1]:
#TODO: Implement an interactive user interface for part a of this exercise.

#Produce a text file with 100 results per query in the format specified above.
def batch(results):
    #TODO: Implement me.    
    pass

# 4. Performance Evaluation 
We will evaluate the performance of our search systems on the basis of five queries. Two of them are avilable to you now for engineering purposes:
- `tübingen attractions`
- `food and drinks`

The remaining three queries will be given to you during our final session on July 22nd. Please be prepared to run your systems and produce a single result file for all five queries live in class. That means you should aim for processing times of no more than ~1 minute per query. We will ask you to send carsten.eickhoff@uni-tuebingen.de that file.

# Grading
Your final projects will be graded along the following criteria:
- 25% Code correctness and quality (to be delivered on this sheet)
- 25% Report (4 pages, PDF, explanation and justification of your design choices)
- 25% System performance (based on how well your system performs on the 5 queries relative to the other teams in terms of nDCG)
- 15% Creativity and innovativeness of your approach (in particular with respect to your search system #2 and user interface #3 innovations)
- 10% Presentation quality and clarity

# Permissible libraries
You can use any general-puprose ML and NLP libraries such as scipy, numpy, scikit-learn, spacy, nltk, but please stay away from dedicated web crawling or search engine toolkits such as scrapy, whoosh, lucene, terrier, galago and the likes. Pretrained models are fine to use as part of your system, as long as they have not been built/trained for retrieval. 
